In [1]:
## utils isimlendirmesi de yapılabilir.
## pipeline'da lazım olacak fonksiyonlar burada tutulur.

In [2]:
# Data Preprocessing & Feature Engineering 

In [3]:
import joblib
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [4]:
def grab_col_names(df, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        df: df
                Değişken isimleri alınmak istenilen df
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """


    # cat_cols, cat_but_car
    cat_cols = [col for col in df.columns if df[col].dtypes == "O"]
    num_but_cat = [col for col in df.columns if df[col].nunique() < cat_th and
                   df[col].dtypes != "O"]
    cat_but_car = [col for col in df.columns if df[col].nunique() > car_th and
                   df[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in df.columns if df[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {df.shape[0]}")
    print(f"Variables: {df.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

## verilen veri setindeki kategorik, sayısal, sayısal gibi görünen kategorik 
# ve kategorik gibi görünen ama kardinal olan değişkenleri döndürür.

In [5]:
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

## kendisine girilen değişkenin alt ve üst eşik değerlerini 
# tanımlanmış yöntem ile hesaplamaktır
# boxplot ile yapmaktadır

In [6]:
def replace_with_thresholds(dataframe, variable, q1=0.25, q3=0.75):
    low_limit, up_limit = outlier_thresholds(dataframe, variable, q1, q3)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

## aykırı değer varsa onun yerine eşik değer değerini koyar

In [7]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

## -türeteceğimiz yeni- kategorik değişkenleri biçimlendirir
# drop_first True dersek one hot encoderi label encoder olarak da kullanmış oluruz
# yani iki değişkenli kategorikleri de encoderdan geçirmiş olacak

In [8]:
# yukarıda tek tek yaptıgımız veri on isleme adımlarının fonksiyonlasması
def diabetes_data_prep(dataframe):
    dataframe.columns = [col.upper() for col in dataframe.columns]

    # Glucose
    dataframe['NEW_GLUCOSE_CAT'] = pd.cut(x=dataframe['GLUCOSE'], bins=[-1, 139, 200], labels=["normal", "prediabetes"])

    # Age
    dataframe.loc[(dataframe['AGE'] < 35), "NEW_AGE_CAT"] = 'young'
    dataframe.loc[(dataframe['AGE'] >= 35) & (dataframe['AGE'] <= 55), "NEW_AGE_CAT"] = 'middleage'
    dataframe.loc[(dataframe['AGE'] > 55), "NEW_AGE_CAT"] = 'old'

    # BMI
    dataframe['NEW_BMI_RANGE'] = pd.cut(x=dataframe['BMI'], bins=[-1, 18.5, 24.9, 29.9, 100],
                                        labels=["underweight", "healty", "overweight", "obese"])

    # BloodPressure
    dataframe['NEW_BLOODPRESSURE'] = pd.cut(x=dataframe['BLOODPRESSURE'], bins=[-1, 79, 89, 123],
                                            labels=["normal", "hs1", "hs2"])

    cat_cols, num_cols, cat_but_car = grab_col_names(dataframe, cat_th=5, car_th=20)

    cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

    df = one_hot_encoder(dataframe, cat_cols, drop_first=True)

    df.columns = [col.upper() for col in df.columns]

    cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=5, car_th=20)

    cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

    replace_with_thresholds(df, "INSULIN")

    X_scaled = StandardScaler().fit_transform(df[num_cols])
    df[num_cols] = pd.DataFrame(X_scaled, columns=df[num_cols].columns)

    y = df["OUTCOME"]
    X = df.drop(["OUTCOME"], axis=1)

    return X, y

In [9]:
# Base Models

In [10]:
def base_models(X,y,scoring="roc_auc"):
    print("Base models....")
    classifiers = [("LR",LogisticRegression()),
                   ("KNN",KNeighborsClassifier()),
                   ("SVC",SVC()),
                   ("CART",DecisionTreeClassifier()),
                   ("RF",RandomForestClassifier()),
                   ("Adaboost",AdaBoostClassifier()),
                   ("GBM",GradientBoostingClassifier()),
                   ("XGBoost",XGBClassifier(use_label_encoder=False,eval_metric="logloss")),
                   ("LightGBM",LGBMClassifier(verbose=-1)),
                   ("CatBoost",CatBoostClassifier(verbose=False))
                  ]
    for name, classifier in classifiers:
        cv_results = cross_validate(classifier,X,y,cv=3,scoring=scoring)
        print(f"{scoring}: {round(cv_results['test_score'].mean(),4)} ({name})")

In [11]:
# Hyperparameter Optimization

In [12]:
# birden fazla modelin birden fazla arama görevini bir fonksiyon ile otomatik şekilde gerçekleşmesi


In [13]:
def hyperparameter_optimization(X,y,cv=3,scoring="roc_auc"):
    knn_params = {"n_neighbors": range(2, 50)}

    cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

    rf_params = {"max_depth": [8, 15, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [15, 20],
             "n_estimators": [200, 300]}

    xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8],
                  "n_estimators": [100, 200]}

    lightgbm_params = {"learning_rate": [0.01, 0.1],
                   "n_estismators": [300, 500]}


    classifiers = [('KNN', KNeighborsClassifier(), knn_params),
               ("CART", DecisionTreeClassifier(), cart_params),
               ("RF", RandomForestClassifier(), rf_params),
               ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), xgboost_params),
               ('LightGBM', LGBMClassifier(verbose=-1), lightgbm_params)]
    
    print("Hyperparameter optimization....")
    best_models = {} # görevi en iyi modelleri tutmak
    for name, classifier, params in classifiers:
        print(f"########### {name} ###########") # calısan modelin adı
        cv_results = cross_validate(classifier,X,y,cv=cv,scoring=scoring) # optimize edilmemis skorun hesaplanması
        print(f"{scoring} (Before): {round(cv_results['test_score'].mean(),4)}") # skorun yazdırılması
        
    # modelin parametre optimizasyonunun yapılması
        gs_best = GridSearchCV(classifier,params,cv=cv,n_jobs=-1,verbose=False).fit(X,y)
    # grid searchten gelen parametrelerle eski modeli set eder
        final_model = classifier.set_params(**gs_best.best_params_)
        
        cv_results = cross_validate(final_model,X,y,cv=cv,scoring=scoring) # optimize edilmiş skorun hesaplanması
        print(f"{scoring} (After): {round(cv_results['test_score'].mean(),4)}") # skorun yazdırılması
        print(f"{name} best params: {gs_best.best_params_}", end="\n\n") # en iyi parametrelerin yazdırılması
        best_models[name] = final_model # sözlüğe aktarılır
    return best_models

In [14]:
# Stacking & Ensemble Learning

In [15]:
def voting_classifier(best_models,X,y):
    print("Voting classifier....")
    voting_clf = VotingClassifier(estimators=[("KNN",best_models["KNN"]),
                                             ("RF",best_models["RF"]),
                                              ("LightGBM",best_models["LightGBM"])],voting="soft").fit(X,y)
    cv_results = cross_validate(voting_clf,X,y,cv=3,scoring=["accuracy","f1","roc_auc"])
    print(f"Accuracy: {cv_results['test_accuracy'].mean()}")
    print(f"F1Score: {cv_results['test_f1'].mean()}")
    print(f"ROC_AUC: {cv_results['test_roc_auc'].mean()}")
    return voting_clf
## hard- en fazla tahmin edilen sınıfı döndürür
## soft- sınıf gerçekleşme olasılıkları üzerinden oylama yapılır